In [1]:
import asyncio
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
import win32com.client as win32
import os
import re
from io import BytesIO
import pandas as pd
import dataframe_image as dfi
import pandas as pd
import random
from pydataquery import DataQuery
from matplotlib.colors import LinearSegmentedColormap
from xbbg import blp
import numpy as np
import io
import pytz
import time
from datetime import datetime, timedelta
import statsmodels.api as sm
from ipywidgets import interact, Dropdown, HBox, VBox, Button, Output, Text, widgets
from IPython.display import display, clear_output, update_display
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import re
import scipy.stats as stats
from adjustText import adjust_text
import warnings
import openpyxl
import pyautogui
import pythoncom
import subprocess
import pygetwindow as gw
import pyodbc
import ast
warnings.filterwarnings("ignore")

In [2]:
try:
    df_original = pd.read_parquet("Markit CDS.parquet")
    
    to_date = datetime.today().date()
    from_date = df_original["close_date"].iloc[-1].date()
    
    from_date_str = from_date.strftime('%m/%d/%Y')
    to_date_str = to_date.strftime('%m/%d/%Y')
    
    conn_str = (
        f'DRIVER={{SQL Server}};'
        f'SERVER=BC-ODS-P1;'
        f'DATABASE=MarkitDB;'
        f'ApplicationIntent=ReadOnly;'
        f'Trusted_Connection=Yes;'
        f'Authentication=ActiveDirectoryIntegrated;'
    )
    
    conn = pyodbc.connect(conn_str)
    
    query1 = f"""
        DECLARE @FromDate DATE = '{from_date_str}';
        DECLARE @ToDate DATE = '{to_date_str}';
        
        SELECT sc.close_date, r.ticker, c.red, c.tier, c.docclause, c.ccy, sc.tenor, sc.spread
        FROM dbo.RedEntities r
        INNER JOIN dbo.MarkitCurves c ON r.red = c.red
        INNER JOIN dbo.MarkitSpreadCurve sc ON c.curve_id = sc.curve_id
        WHERE sc.close_date >= @FromDate AND sc.close_date <= @ToDate;
    """
    
    df_new = pd.read_sql(query1, conn)
    conn.close()

    df_old = df_original[df_original["close_date"]<pd.to_datetime(df_original["close_date"].iloc[-1])]

    df1 = pd.concat([df_old, df_new])
    df1 = df1.drop_duplicates()
    df1.to_parquet("Markit CDS.parquet")
except:
    df1 = pd.read_parquet("Markit CDS.parquet")
    hello = 1

In [3]:
markit_cds = list(set(df1["red"]))
markit_df1 = df1.copy()

res_codes = { "Full Restructuring": "CR14", "Modified Restructuring": "MR14",
          "Modified-Modified Restructurin": "MM14", "No Restructuring": "XR14"}

for rating_col in ["IG","HY","EUR IG","EUR HY","SNRFIN"]:

    dq = pd.read_excel("CDX Members.xlsx", sheet_name=rating_col)
        
    dq["Restructuring"] = dq["Restructuring"].apply(lambda x: res_codes[x])
    dq = dq[dq["Actual RED Code"].isin(markit_cds)].reset_index(drop=True).copy()
    dq.rename(columns={"Actual RED Code": "red"}, inplace=True)
    dq["Eqty"] = dq["Issuer Equity"]
    
    #################################
    df1 = markit_df1.copy()
    cds_df1 = df1[(df1["tier"]=="SNRFOR") & (df1["ccy"]=="USD") & (df1["tenor"]!="Spot") &\
        (df1["docclause"].str.endswith("14"))].drop(["tier","ccy"],axis=1).copy()
    
    ########## SNRFOR is the only sr unsecured option
    
    # cds_df["close_date"] = pd.to_datetime(cds_df["close_date"])
    # list_active_cos = set(cds_df[(cds_df["close_date"]==cds_df["close_date"].max())]["red"])
    # cds_df = cds_df[cds_df["red"].isin(list_active_cos)].reset_index(drop=True).copy()
    
    #################################
    cds_df = cds_df1.copy()
    
    df = pd.merge(left=cds_df, right=dq, on="red", how="inner")
    df = df[df["docclause"]==df["Restructuring"]].drop(["RED Code","Restructuring"],axis=1).reset_index(drop=True).copy()
    cds_df = df[["close_date","ticker","red","tenor","spread","Eqty","Bond Name"]].copy()
    cds_df.rename(columns={"Eqty": "Eqty Name"}, inplace=True)
    
    cds_df["tenor"] = cds_df["tenor"].apply(lambda x: eval(x.replace("y","*1").replace("m","*(1/12)")))
    cds_df = cds_df[~pd.isna(cds_df["Eqty Name"])]
    cds_df = cds_df[~(cds_df["Eqty Name"].str.startswith("#"))].reset_index(drop=True).copy()


##############################################################################################################
    #############################################################
    issuers = list(sorted(set(cds_df["Eqty Name"])))
    last_dt = max(cds_df["close_date"])
    
    all_curves = None
    
    try:
        for issuer in issuers: ################################################################## issuers instead of issuers1 when going live
            # issuer = issuers[0]
            df = cds_df[cds_df["Eqty Name"]==issuer]
            df = pd.pivot_table(df, values="spread", index="close_date", columns ="tenor")
            
            if not last_dt in df.index:
                df.loc[last_dt] = [np.nan] * len(df.columns)
            df = df.sort_index().ffill().copy()
            
            df[f"{issuer}_curve"] = [np.nan] * len(df)
        
            for idx in df.index:
                # idx = df.index[0]
                curve = df.loc[[idx],:].dropna(axis=1)
                curve = curve.iloc[:,:-1].copy()
                x = list((curve.columns))
                x2 = [item**2 for item in x]
                X = np.column_stack([x, x2])
                Y = list(curve.iloc[0])
        
                if len(X) > 2:
                    model = sm.OLS(Y, sm.add_constant(X)).fit()
                    df.loc[idx,f"{issuer}_curve"] = str([model.params[0], model.params[1], model.params[2]])
                    
                    # x_pred = list(np.linspace(min(x), max(x),100))
                    # x_pred2 = [item**2 for item in x_pred]
                    # X_pred = np.column_stack([x_pred, x_pred2])
                    # y_pred = model.predict(sm.add_constant(X_pred))
                    
                    # plt.plot(x, Y)
                    # plt.plot(x_pred, y_pred)
                    # title = f"{issuer} on {str(idx.date())}"
                    # plt.title(title)
                    # # plt.savefig(f"Curve Plots v2/{title}.png")
                    # plt.show()
                    # plt.close()
                    
                else:
                    df.loc[idx,f"{issuer}_curve"] = str([np.nan, np.nan, np.nan])
        
            all_curves = pd.concat([all_curves,df.iloc[:,[-1]]],axis=1)
        all_curves.to_excel("All CDS Curves.xlsx")
    except:
        all_curves = pd.read_excel("All CDS Curves.xlsx",index_col=0, parse_dates=True)
        
    all_curves1 = all_curves.copy()
    all_curves = all_curves.T.copy()
    all_curves.index = [item.split("_")[0] for item in all_curves.index]
    all_curves.index.name = "Eqty Name"
    all_curves.columns = [f'CDS_{item.date()}' for item in all_curves.columns]
    all_curves = all_curves.reset_index(drop=False).copy()
    temp_cds = all_curves.copy()

##############################################################################################################

    ################ This will fetch latest list of all bonds in this index
    
    # ##########################################################################################
    # bql_list = []
    
    # fl = [f"""=BQL("filter(bonds(['""", f""" Equity']), payment_rank=='Sr Unsecured' AND crncy=='USD')", "id_isin, id_cusip, name, maturity, amt_outstanding")"""]
    # for item in issuers:
    #     bql_list += [(fl[0] + item + " " + fl[1])]
        
    # bql_list = [item.replace("\\", "") for item in bql_list]
    
    # workbook = openpyxl.load_workbook(r"J:\\HY Basis Data.xlsx")
    # sheet = workbook.active
    
    # for row in sheet.iter_rows():
    #     for cell in row:
    #         cell.value = None
    
    # start_col = 1
    # for item in bql_list:
    #     cell = sheet.cell(row=2, column=start_col)
    #     cell.value = item
    #     start_col += 6
    # workbook.save(r"J:\\HY Basis Data.xlsx")
    
    # file_path = r"J:\\HY Basis Data.xlsx"
    # window_title = "HY Basis Data - Excel"
    
    # subprocess.Popen(['start', 'excel', file_path], shell=True)
    # time.sleep(5)
    
    # excel_windows = [window for window in gw.getWindowsWithTitle('Excel')]
    
    # for window in excel_windows:
    #     if window_title in window.title:
    #         # time.sleep(0.5)
    #         window.activate()
    #         break
    
    # time.sleep(45)
    # pyautogui.hotkey('ctrl', 's')
    # time.sleep(1)
    # pyautogui.hotkey('alt', 'f4')
    
    # time.sleep(1.5)

##############################################################################################################

    
    issuer_dict = dict(zip(dq["Issuer Equity"],[item.split(" ")[0] for item in dq["Bond Name"]]))
    ############################################################################ Now get those bonds
    # hy = pd.read_excel(r"J:\\HY Basis Data.xlsx",skiprows=1)
    hy = pd.read_excel("All Bonds.xlsx", sheet_name=rating_col)
    
    all_df = None
    
    for i in range(len(hy.columns))[::6]:
        x = hy.iloc[:,i:i+6].dropna().copy()
        if len(x) > 0:
            x.columns = ['ID','ISIN', "CUSIP", 'Name', 'Maturity','Amt']
            x["Bond Name"] = [issuer_dict[issuers[(int(i/6))]]] * len(x)
            x["Check Col"] = x.apply(lambda row: row["Name"].split(" ")[0]==row["Bond Name"],axis=1)
            x = x[x["Check Col"]].drop("Check Col",axis=1)
            all_df = pd.concat([all_df, x])
    
    all_df['Time'] = round(((pd.to_datetime(all_df['Maturity'])-datetime.now()).dt.days/365),2)
    all_df = all_df[all_df["Time"]>=0]
    all_df = all_df[(all_df['Time']>=4) & (all_df['Time']<=10)]
    all_df = all_df[all_df['Amt']>=300*10**6]
    all_df = all_df.reset_index(drop=True)
    
    ###################################### 144A and REGS
    
    blist = [f'/isin/{item}@BGN' for item in list(all_df["ISIN"])]
    blist = blp.bdp(tickers=blist, flds=["144A_FLAG","IS_REG_S"])
    blist.index = [item.rsplit("/",1)[1].split("@")[0] for item in blist.index]
    
    blist.columns = ["144A","REGS"]
    blist.index.name = "ISIN"
    blist = blist.reset_index()
    blist["REGS_144A"] = blist.apply(lambda row: f'{row["REGS"]}_{row["144A"]}',axis=1)
    order = ["N_N", "Y_N", "Y_Y", "N_Y"]
    
    all_df = pd.merge(left=all_df, right=blist, on="ISIN", how="outer")
    all_df = all_df[[item for item in all_df.columns if not item in ["144A","REGS"]]]
    
    
    df2 = all_df.copy()
    df2["REGS_144A"] = pd.Categorical(df2["REGS_144A"], categories=order, ordered=True)
    df2 = df2.sort_values(by="REGS_144A")
    df2 = df2[~df2["Name"].duplicated(keep='first')].drop(["REGS_144A","Time"],axis=1).reset_index(drop=True).copy()
    # df2.to_excel("HY Bonds List.xlsx")
    
    ##################################################################################
    
    # df2 = pd.read_excel("HY Bonds List.xlsx", index_col=0, parse_dates=True)
    
    t = [f"/isin/{item}@BGN" for item in list(df2["ISIN"])]
    bbg = blp.bdh(tickers=t, flds="BLOOMBERG_MID_G_SPREAD", start_date=datetime.now()-timedelta(days=365*5))
    # bbg.to_parquet("Test1.parquet")
    # bbg = pd.read_parquet("Test1.parquet")
    
    bbg1 = bbg.copy()
    new = []
    for item in bbg1.columns:
        new += ["BBG_" + item[0].replace("/isin/","").replace("@BGN","")]
    bbg1.columns = new
    bbg1.index = pd.to_datetime(bbg1.index)
    
    ########################### choose bbg or dq sprds
    bbg1.columns = [item.split("_")[1] for item in bbg1.columns]
    bbg1.index = [f"Sprd_{str(item.date())}" for item in bbg1.index]
    bbg1 = bbg1.T.copy()
    bbg1.index.name = "ISIN"
    bbg1 = bbg1.reset_index(drop=False).copy()

##############################################################################################################

    # df2 = pd.read_excel("HY Bonds List.xlsx", index_col=0, parse_dates=True)
    
    # # # ########################################## DQ Spreads
    # # ############################################################
    
    # # spd = {"Spd": ["DB(CREDIT,HY,BOND,",",STW)"],
    # #        # "PECS": ["DB(FIR,CUSIP,",",PECS)"]
    # #       }
    # # dq_hy_dict = {}
    
    # # for item in list(df2["CUSIP"]):
    # #     for key, value in spd.items():
    # #         dq_hy_dict[f'{item}_{key}'] = value[0] + item + value[1]
    
    
    # # ######################### Getting HY Data from BQL
    
    # # try:
    # #     dq = DataQuery(
    # #         # client_id='jbAIMF2Tkp0JO3sc',
    # #         client_secret='d7qfzgt55pddjs352sgxosFyI4t2eja07k7opbi6wg9oqjc1OjkdAksn1btmnugeMjchcx2vwTsJupw',
    # #         # calendar = 'CAL_USBANK',
    # #     )
        
    # #     job = dq.create_job(expressions = list(dq_hy_dict.values()))
    # #     dq.start_date = 'TODAY-300bd'
    # #     job.execute()
    # #     df = job.to_pivot_table()
    # #     df = df.T
    # #     df.index.name = 'Date'
    # #     df.index = pd.to_datetime(df.index, format='%Y%m%d').date
    # #     df.columns.name = None
        
    # #     df1 = pd.DataFrame()
    # #     for key in dq_hy_dict:
    # #         df1[key] = df[dq_hy_dict[key]]
        
    # #     df1 = df1[list(dq_hy_dict.keys())].copy()
    # #     clear_output(wait=False)
    # #     dfa = df1.dropna(axis=1, how='all').copy()
    # #     dfa.to_excel("HY Bond Sprd Data.xlsx")
    # # except:
    # #     dfa = pd.read_excel("HY Bond Sprd Data.xlsx", index_col=0, parse_dates=True)
    # # dfa = pd.read_excel("HY Bond Sprd Data.xlsx", index_col=0, parse_dates=True)
    # # dft = dfa.copy()
    # # dft.columns = [item.replace("_Spd","") for item in dft.columns]
    # # data = dict(zip(list(df2["CUSIP"]), list(df2["ISIN"])))
    # # dft.columns = ["DQ_" + data[item] for item in dft.columns]
    
    # ########################################## BBG Spreads
    
    # t = [f"/isin/{item}@BGN" for item in list(df2["ISIN"])]
    # bbg = blp.bdh(tickers=t, flds="BLOOMBERG_MID_G_SPREAD", start_date=datetime.now()-timedelta(days=400))
    # # bbg.to_parquet("Test1.parquet")
    # # bbg = pd.read_parquet("Test1.parquet")
    
    # bbg1 = bbg.copy()
    # new = []
    # for item in bbg1.columns:
    #     new += ["BBG_" + item[0].replace("/isin/","").replace("@BGN","")]
    # bbg1.columns = new
    # bbg1.index = pd.to_datetime(bbg1.index)
    
    # ########################### choose bbg or dq sprds
    # bbg1.columns = [item.split("_")[1] for item in bbg1.columns]
    # bbg1.index = [f"Sprd_{str(item.date())}" for item in bbg1.index]
    # bbg1 = bbg1.T.copy()
    # bbg1.index.name = "ISIN"
    # bbg1 = bbg1.reset_index(drop=False).copy()

##############################################################################################################

    df3 = df2.copy()
    for dt in pd.to_datetime(bbg.index):
        df3[f'Mat_Time_{dt.date()}'] = [((pd.to_datetime(item) - dt).days/365) for item in df3["Maturity"]]
        
    df3_connect = dict(zip([item.split(" ")[0] for item in dq["Bond Name"]], dq["Eqty"]))
    df3["Eqty Name"] = df3["Bond Name"].apply(lambda x: df3_connect[x])
    df3 = df3.drop("Bond Name",axis=1)
    ######################################################################
    
    df4 = pd.merge(left = df3, right = temp_cds, on="Eqty Name", how="outer")
    df4 = df4[~pd.isna(df4["ISIN"])].reset_index(drop=True).copy()
    dt_list = [item.replace("CDS_","") for item in df4.columns if item.startswith("CDS_")]
    
    df4A = df4.copy()
    
    for dt in dt_list:
        if f"Mat_Time_{dt}" in df4A.columns and f"CDS_{dt}" in df4A.columns:
            df4A[f'Mat_Matched_CDS_{dt}'] = df4A[f'CDS_{dt}'].apply(lambda x: ast.\
                literal_eval(x)[0] if isinstance(x,str) else np.nan) +\
            df4A[f'Mat_Time_{dt}'].apply(lambda x: x) *\
            df4A[f'CDS_{dt}'].apply(lambda x: ast.literal_eval(x)[1] if isinstance(x,str) else np.nan) +\
            df4A[f'Mat_Time_{dt}'].apply(lambda x: x**2) *\
            df4A[f'CDS_{dt}'].apply(lambda x: ast.literal_eval(x)[2] if isinstance(x,str) else np.nan)
    
    df4A = df4A[[col for col in df4A.columns if not "_" in col] +\
        [col for col in df4A.columns if col.startswith("Mat_Matched_CDS_")]]

##############################################################################################################

    df5 = pd.merge(left=df4A, right=bbg1, how='outer', on="ISIN")
    last_update = max(dt_list)
    for dt in dt_list:
        try:
            df5[f"Basis_{dt}"] = df5[f"Mat_Matched_CDS_{dt}"] - df5[f"Sprd_{dt}"]
        except:
            hello = 1
    l1 = list(set([col for col in df5.columns if f'{last_update}' in col] + [col for col in df5.columns if 'Basis' in col] ))
    l2 = [item for item in l1 if f'{last_update}' in item] + [item for item in l1 if not f'{last_update}' in item]
    df5 = df5[[col for col in df5.columns if not "_" in col] + l2]
    df5 = df5[~pd.isna(df5[f"Sprd_{last_update}"])].reset_index(drop=True).copy()
    
    red_eqty_dict1 = dict(zip(list(dq["Eqty"]), list(dq["red"])))
    
    df5["CDS RED Code"] = df5["Eqty Name"].apply(lambda x: red_eqty_dict1[x])
    
    ####################################### Zscore and 1Y High/Low Calc
    
    dfz = df5[["ISIN"] + [item for item in df5.columns if "Basis" in item]]
    dfz = dfz.set_index("ISIN").T
    dfz.index = [pd.to_datetime(item.split("_")[1]).date() for item in dfz.index]
    dfz = dfz.sort_index()
    
    all_z_df = None
    for period in [3*22, 6*22, 12*22]:
        x = ((dfz-dfz.rolling(period, min_periods = int(0.6*period)).mean())/dfz.rolling(period,\
                      min_periods = int(0.6*period)).std()).iloc[[-1],:].copy()
        # x = ((dfz-dfz.rolling(period, min_periods = int(0.5*period)).mean())/dfz.rolling(period,\
        #               min_periods = int(0.5*period)).std()).iloc[[-1],:].copy()
        x.index = [f'ZScore_{int(period/22)}M']
        all_z_df = pd.concat([all_z_df , x])
    
    max_1y = pd.DataFrame(dfz.iloc[-12*22:,:].max()).T
    max_1y.index = [f'Basis 1Y High']
    min_1y = pd.DataFrame(dfz.iloc[-12*22:,:].min()).T
    min_1y.index = [f'Basis 1Y Low']
    
    all_z_df = pd.concat([all_z_df , max_1y, min_1y])
    all_z_df = all_z_df.T
    all_z_df = all_z_df.reset_index()
    
    df6 = pd.merge(left=df5[[col for col in df5.columns if not col.startswith("Basis_") or dt in col]],
                   right=all_z_df, on="ISIN", how="outer")
    df6 = df6[["Name", "ISIN", "Eqty Name", "CDS RED Code", f"Basis_{last_update}",
               f"Sprd_{last_update}", f"Mat_Matched_CDS_{last_update}"] + list(df6.columns)[-5:]]
    
    
    ################################### Add PX
    px_list = [f"/isin/{i}@BGN" for i in list(df6["ISIN"])]
    px = blp.bdp(tickers=px_list, flds="px_last")
    # px.to_parquet("Test2.parquet")
    # px = pd.read_parquet("Test2.parquet")
    
    px1 = px.copy()
    px1.index = [item.replace("/isin/","").replace("@BGN","") for item in px1.index]
    px1.columns = ["Price"]
    px1.index.name ="ISIN"
    px1 = px1.reset_index().copy()
    df6 = pd.merge(left=df6, right=px1, how="outer", on="ISIN")
    
    
    ################################################## Adding IG/HY
    blist = [f"/isin/{item}@BGN" for item in list(df6["ISIN"])]
    ig_hy = blp.bdp(tickers=blist, flds="BB_COMPSTE_RATING_IG_HY_INDCTR")
    ig_hy["ISIN"] = [item.replace("/isin/","").replace("@BGN","") for item in ig_hy.index]
    ig_hy.columns = ["IG_HY","ISIN"]
    ig_hy = ig_hy.reset_index(drop=True).copy()
    df6 = pd.merge(left=df6, right=ig_hy, how="outer", on="ISIN")
    df6 = df6.sort_values(by="Eqty Name").reset_index(drop=True).copy()
    
    ################################################## 
    
    df7 = df6.copy()
    df7["Time"] = df7["Name"].apply(lambda x: x.split("/")[-1])
    df7 = df7.sort_values(by=["Eqty Name","Time"],ascending=True).\
        reset_index(drop=True).drop(["Eqty Name","Time","ISIN"],axis=1).copy()
    df7 = df7.set_index("Name")
    df7.columns = df7.columns.str.replace(f'_{last_update}',"")
    df7.index.name = f'{pd.to_datetime(last_update).strftime("%d-%b")}'
    df7 = round(df7,2)
    
    df7 = df7[["IG_HY","CDS RED Code","Price","Sprd","Mat_Matched_CDS","Basis",
               "Basis 1Y Low","Basis 1Y High","ZScore_3M","ZScore_6M","ZScore_12M"]]
    
    df7["Name"] = [item.split(" ")[0] for item in df7.index]
    df7["Year"] = [eval(item.split("/")[-1]) for item in df7.index]
    df7 = df7.sort_values(by=["Name","Year"], ascending=True).drop(["Name","Year"],axis=1).copy()

##############################################################################################################
                        
    # for rating_col in ["IG", "HY"]:
        
    # df1 = df7[df7["IG_HY"]==rating_col].drop("IG_HY", axis=1).copy()
    
    df1 = df7.drop("IG_HY", axis=1).copy()
    df1["Issuer"] = [item.split(" ")[0] for item in df1.index]
    
    num_issuers = len(set([item.split(" ")[0] for item in df1.index]))
    
    issuers_per_table = 4
    issuers_list = list(sorted(set(df1["Issuer"])))
    
    for i in range(int(num_issuers/issuers_per_table)+1):
        globals()[f'df_{i}'] = df1[df1["Issuer"].isin(issuers_list[issuers_per_table*i:issuers_per_table*i+issuers_per_table])].copy()
    
    modified_dfs = []
    
    for i in range(int(num_issuers / issuers_per_table) + 1):
        df = globals()[f'df_{i}'].reset_index().copy()
        new_df = pd.DataFrame(columns=df.columns)
        
        for issuer in df['Issuer'].unique():
            issuer_rows = df[df['Issuer'] == issuer]
            new_df = pd.concat([new_df, issuer_rows], ignore_index=True)
            empty_row = pd.Series([''] * len(df.columns), index=df.columns)
            new_df = pd.concat([new_df, pd.DataFrame([empty_row])], ignore_index=True)
        globals()[f'df_{i}'] = new_df.set_index(new_df.columns[0]).drop("Issuer",axis=1).copy()
    
    def color_negative_red_positive_green_basis(col: pd.Series):
        if col.empty:
            return ['' for _ in col]
        float_col = col[col.apply(lambda x: isinstance(x, float))]
        min_val, max_val = float_col.min(), float_col.max()
        def value_to_color(val):
            if not isinstance(val, float):
                return ''
            if val < 0 and min_val < 0:
                frac = val/ min_val
                frac = max(min(frac,1),0)
                r = int(255 - (255 - 87) * frac)
                g = int(255 - (255 - 187) * frac)
                b = int(255 - (255 - 138) * frac)
                a = 0.75
                return f'background-color: rgb({r},{g},{b},{a})'
            elif val > 0 and max_val > 0:
                frac = val/ max_val
                frac = max(min(frac,1),0)
                r = int(255 - (255 - 230) * frac)
                g = int(255 - (255 - 135) * frac)
                b = int(255 - (255 - 115) * frac)
                a = 0.75
                return f'background-color: rgb({r},{g},{b},{a})'
            return ''
        return [value_to_color(v) for v in col]
                
    
    def add_vertical_lines(df):
        styles = pd.DataFrame('', index=df.index, columns=df.columns)
        for i in range(1, len(df.columns), 2):
            styles.iloc[:, i] = 'border-right: 8px solid black;'
        return styles
        
    def style_rich_cheap_basis(df):
        styled_df = df.style.apply(color_negative_red_positive_green_basis, subset=df.columns[-3:], axis=0)
        styled_df = styled_df.format(precision=2)
        styled_df = styled_df.format(precision=0, subset=df.columns[2:7])
        styled_df = styled_df.format(precision=1, subset=df.columns[-3:])
        styled_df = styled_df.set_properties(**{'text-align': 'center'})
        styled_df = styled_df.applymap(lambda val: 'font-weight: bold', subset=pd.IndexSlice[:, df.columns[4]])
        return styled_df
        
    for i in range(int(num_issuers / issuers_per_table) + 1):
        df = globals()[f'df_{i}'].copy()
        index_counts = {}
        new_index = []
        
        for idx in df.index:
            if idx in index_counts:
                index_counts[idx] += 1
                # new_index.append(f"{idx}_{index_counts[idx]}")
                new_index.append(f"{idx}{'_' * index_counts[idx]}{pd.to_datetime(last_update).strftime("%d-%b")}")
    
            else:
                count = df.index.tolist().count(idx)
                if count > 1:
                    index_counts[idx] = 1
                    new_index.append(f"{idx}_{pd.to_datetime(last_update).strftime("%d-%b")}")
                else:
                    new_index.append(idx)
        df.index = new_index
        df.index.name = f'{pd.to_datetime(last_update).strftime("%d-%b")}'
        globals()[f'display_df_{i}'] = style_rich_cheap_basis(df.iloc[:-1,:])
        globals()[f'display_df_{i}'] = globals()[f'display_df_{i}'].set_table_styles(
            [
                {'selector': 'tr', 'props': [('width', '8px')]},
                {'selector': 'th, td', 'props': [('line-height', '8px')]}
            ],)
    
        dfi.export(globals()[f'display_df_{i}'] , f'Plots/{rating_col}/{rating_col}_{i}.png', table_conversion='chrome', dpi=100)
    
    import os
    import win32com.client
    import glob
    import pandas as pd
    import dataframe_image as dfi
    from io import BytesIO
    import tempfile
    
    # html_body = "<html><body>"
    
    # outlook = win32com.client.Dispatch("Outlook.Application")
    # mail = outlook.CreateItem(0)
    
    # folder_path = f'J:\\53. Live HY Basis\\Plots\\{rating_col}'
    # png_files = glob.glob(os.path.join(folder_path, '*.png'))
    # png_files = [item for item in png_files if f"{rating_col}_" in item]
    # png_files.sort(key=lambda x: int(os.path.splitext(os.path.basename(x))[0].split("_")[1]))
    
    # for png_file in png_files:
    #     image_cid = os.path.basename(png_file).split('.')[0]    
    #     attachment = mail.Attachments.Add(Source=png_file)
    #     attachment.PropertyAccessor.SetProperty("http://schemas.microsoft.com/mapi/proptag/0x3712001F", image_cid)
    #     html_body += f'<img src="cid:{image_cid}"><br><br><br>'
    
    # html_body += '</body></html>'
    
    # mail.HTMLBody = html_body
    # mail.Subject = f'{rating_col} Basis (auto)'
    # mail.CC = 'vasu.sharma@iiicm.com'
    # # mail.To = 'spyros.michas@iiicm.com'
    # mail.Send()
        
    # print("Sent")

    html_body = "<html><body>"
    
    outlook = win32com.client.Dispatch("Outlook.Application")
    mail = outlook.CreateItem(0)
    
    folder_path = f'J:\\53. Live HY Basis\\Plots\\{rating_col}'
    png_files = glob.glob(os.path.join(folder_path, '*.png'))
    png_files = [item for item in png_files if f"{rating_col}_" in item]
    png_files.sort(key=lambda x: int(os.path.splitext(os.path.basename(x))[0].split("_")[1]))
    
    for png_file in png_files:
        raw_name = os.path.splitext(os.path.basename(png_file))[0]
        image_cid = raw_name.replace(" ", "_")
        attachment = mail.Attachments.Add(Source=png_file)
        attachment.PropertyAccessor.SetProperty("http://schemas.microsoft.com/mapi/proptag/0x3712001F", image_cid)
        html_body += f'<img src="cid:{image_cid}"><br><br><br>'
    
    html_body += '</body></html>'
    
    mail.HTMLBody = html_body
    mail.Subject = f'{rating_col} Basis (auto)'
    mail.CC = 'vasu.sharma@iiicm.com'
    mail.To = 'spyros.michas@iiicm.com'
    mail.Send()
    
    print("Sent")


Sent


Sent


Sent


Sent


Sent
